<a href="https://colab.research.google.com/github/PabloNicolasCassina/Workshop1_homework/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [14]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [15]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [1]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [9]:
# to do: homework :)

#Question 1: What is the sum of the outputs of the generator for limit = 5?

limit = 5


generator = square_root_generator(limit)
sum = 0.0

for number in generator:
  sum = sum + number

print(sum)

8.382332347441762


In [12]:
#Question 2: What is the 13th number yielded
limit = 13


generator = square_root_generator(limit)
for _ in range(13):
    number = next(generator)

print("El decimotercer número generado es:", number)


El decimotercer número generado es: 3.605551275463989


In [26]:
#2. Append a generator to a table with existing data

#a. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
import dlt

generators_pipeline = dlt.pipeline(destination="duckdb", dataset_name = "people")

generator = people_1()

info = generators_pipeline.run(generator,
                               table_name="people1",
                               write_disposition="replace")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707975357.9155734 is LOADED and contains no failed jobs


In [29]:
#total_age

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

t1 = conn.sql("SELECT * FROM people1").df()
display(t1)

total_age = conn.sql("SELECT SUM(age) AS total_age FROM people1")

print(total_age)

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707975357.9155734,udDVu+gOJz9jeg
1,2,Person_2,27,City_A,1707975357.9155734,UlDCpd93r5FQDA
2,3,Person_3,28,City_A,1707975357.9155734,7wTGsqDXcwXnTA
3,4,Person_4,29,City_A,1707975357.9155734,Fkpg+Dnfh8YTLw
4,5,Person_5,30,City_A,1707975357.9155734,+NBdaXJL9LVzqA


┌───────────┐
│ total_age │
│  int128   │
├───────────┤
│       140 │
└───────────┘



In [30]:
#Append the second generator to the same table as the first.

import dlt

generators_pipeline = dlt.pipeline(destination="duckdb", dataset_name = "people")

generator = people_2()

info = generators_pipeline.run(generator,
                               table_name="people1",
                               write_disposition="append")

In [31]:
#view

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

t1 = conn.sql("SELECT * FROM people1").df()
display(t1)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707975357.9155734,udDVu+gOJz9jeg,None
1,2,Person_2,27,City_A,1707975357.9155734,UlDCpd93r5FQDA,None
2,3,Person_3,28,City_A,1707975357.9155734,7wTGsqDXcwXnTA,None
3,4,Person_4,29,City_A,1707975357.9155734,Fkpg+Dnfh8YTLw,None
4,5,Person_5,30,City_A,1707975357.9155734,+NBdaXJL9LVzqA,None
5,3,Person_3,33,City_B,1707975656.9971845,5yzwK7VktjJfKg,Job_3
6,4,Person_4,34,City_B,1707975656.9971845,8TNnxpdihnuzZQ,Job_4
7,5,Person_5,35,City_B,1707975656.9971845,hYvFXbLuDMF9vw,Job_5
8,6,Person_6,36,City_B,1707975656.9971845,D336qDcpcCU2Rg,Job_6
9,7,Person_7,37,City_B,1707975656.9971845,1swFdwIq+sghsw,Job_7


In [32]:
#After correctly appending the data, calculate the sum of all ages of people

total_age = conn.sql("SELECT SUM(age) AS total_age FROM people1")

print(total_age)

┌───────────┐
│ total_age │
│  int128   │
├───────────┤
│       353 │
└───────────┘



In [42]:
#Re-use the generators from Exercise 2.

#A table's primary key needs to be created from the start,
#so load your data to a new table with primary key ID.

import dlt
import duckdb

generators_pipeline = dlt.pipeline(destination="duckdb", dataset_name = "people")

generator = people_1()

info = generators_pipeline.run(generator,
                               table_name="people1",
                               write_disposition="replace",
                               primary_key = "id")


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

t1 = conn.sql("SELECT * FROM people1").df()
display(t1)


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707976416.234754,qxG4m8vSPhWkCg,None
1,2,Person_2,27,City_A,1707976416.234754,+FTsuiG25bmPCw,None
2,3,Person_3,28,City_A,1707976416.234754,jFSC5iokZypEFQ,None
3,4,Person_4,29,City_A,1707976416.234754,Mt3Yv3GxrEEDBQ,None
4,5,Person_5,30,City_A,1707976416.234754,Fd7oZ6Y1cpOxYQ,None


In [43]:
#load the second one with merge. Since they have overlapping IDs,
#some of the records from the first load should be replaced by the ones from the second load.

import dlt
import duckdb

generators_pipeline = dlt.pipeline(destination="duckdb", dataset_name = "people")

generator = people_2()

info = generators_pipeline.run(generator,
                               table_name="people1",
                               write_disposition="merge",
                               primary_key = "id")


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

t1 = conn.sql("SELECT * FROM people1").df()
display(t1)

#Age of ID 3 is 33

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707976416.234754,qxG4m8vSPhWkCg,None
1,2,Person_2,27,City_A,1707976416.234754,+FTsuiG25bmPCw,None
2,5,Person_5,35,City_B,1707976478.5660758,qtOmmRV/AM7N8w,Job_5
3,7,Person_7,37,City_B,1707976478.5660758,aNPiwrPpdVWhcA,Job_7
4,8,Person_8,38,City_B,1707976478.5660758,W5KTKDsDLYU0Tw,Job_8
5,4,Person_4,34,City_B,1707976478.5660758,3igMFZtFxudcEA,Job_4
6,3,Person_3,33,City_B,1707976478.5660758,c0FpQRfvCmeFeg,Job_3
7,6,Person_6,36,City_B,1707976478.5660758,iIqlS8FRGuzY0A,Job_6


In [44]:
total_age = conn.sql("SELECT SUM(age) AS total_age FROM people1")

print(total_age)

┌───────────┐
│ total_age │
│  int128   │
├───────────┤
│       266 │
└───────────┘



Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX